### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 10.  10.  10.  10.]
 [ 10.   4.   9.  10.]
 [  8.   9.  10.   5.]
 [  9.   8.  10.  10.]
 [ 10.   5.   9.   9.]
 [  6.   4.  10.   6.]
 [  9.   8.  10.   9.]
 [ 10.   5.   9.   8.]
 [  7.   8.  10.   8.]
 [  9.   5.   9.   7.]
 [  9.   8.  10.   8.]
 [  9.  10.  10.   9.]
 [ 10.   9.  10.   8.]
 [  5.   8.   5.   8.]
 [ 10.   8.  10.  10.]
 [  9.   9.  10.  10.]
 [  9.   8.   8.   8.]
 [ 10.   8.   1.  10.]
 [  5.   6.  10.  10.]
 [  8.   7.  10.   7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [34]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]# number of rows in the matrix
    n_movies =ratings_mat.shape[1] # number of movies in the matrix
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))# total number of ratings in the matrix
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users,latent_features)# user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features,n_movies)# movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for itr in range(iters):
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i,j]>0:
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    error=ratings_mat[i,j]- np.dot(user_mat[i,:],movie_mat[:,j])
                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum+=error**2
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i,k]+=learning_rate*2*error*movie_mat[k,j]
                        movie_mat[k,j]+=learning_rate*2*error*user_mat[i,k]
        # print results for iteration
        print(itr+1,"\t\t",sse_accum/num_ratings)
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [35]:
user_mat, movie_mat = FunkSVD(ratings_mat,4,0.005,10)# use your function with 4 latent features, lr of 0.005 and 10 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 44.672050362
2 		 16.2702554603
3 		 4.69401805825
4 		 2.92889561103
5 		 2.71310108698
6 		 2.66255367731
7 		 2.63499184845
8 		 2.60942818093
9 		 2.58175298177
10 		 2.55088344348


In [36]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 10.16756173   8.81721933  10.64930437  10.08943022]
 [  8.96304458   7.41260416   9.13687624   8.54892663]
 [  7.74560622   6.8562649    8.25895923   7.88529465]
 [  9.25773911   8.68218955   9.85527987   9.6586638 ]
 [  8.71703268   7.22588618   8.9667321    8.55349952]
 [  7.02673546   5.86186677   7.09441192   6.90999249]
 [  9.04439429   8.22437502   9.51612718   9.32924473]
 [  8.32026788   6.98572818   8.75879662   8.13149858]
 [  8.59661835   7.2282194    8.97809908   8.45435034]
 [  7.82912797   6.4436259    8.18605646   7.66431294]
 [  9.19562771   7.55487207   9.36120711   8.76754873]
 [  9.47233835   8.48317546  10.07165279   9.5755409 ]
 [  9.35171193   8.18159075   9.68657995   9.23295502]
 [  6.24844137   6.23990125   6.6456672    6.85725748]
 [  9.76380976   8.47810933  10.33586006   9.6658456 ]
 [  9.45412286   8.69335691  10.06997266   9.95613194]
 [  8.36506458   7.36250499   8.66229213   8.23684446]
 [  7.16634732   6.16066336   7.27426843   7.25083997]
 [  8.1626

**Write your summary here.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [37]:
user_mat, movie_mat = FunkSVD(ratings_mat,4,0.005,250)#use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 43.0913652959
2 		 15.2135634054
3 		 4.20921177026
4 		 2.76002941736
5 		 2.60946933347
6 		 2.56195457587
7 		 2.52411946478
8 		 2.48460124295
9 		 2.4413677313
10 		 2.39388942513
11 		 2.34186624681
12 		 2.28505281041
13 		 2.22327101312
14 		 2.15644412338
15 		 2.08462659954
16 		 2.0080273178
17 		 1.92702611546
18 		 1.8421827869
19 		 1.75423697524
20 		 1.66409712858
21 		 1.57281684897
22 		 1.48155763624
23 		 1.39153834835
24 		 1.30397369772
25 		 1.22000653959
26 		 1.1406409883
27 		 1.06668468745
28 		 0.998708126629
29 		 0.9370265045
30 		 0.881705773861
31 		 0.832590248348
32 		 0.789345732066
33 		 0.751510416077
34 		 0.718545968914
35 		 0.689882904576
36 		 0.664956680156
37 		 0.643233309912
38 		 0.624225085386
39 		 0.607498074866
40 		 0.592673497363
41 		 0.579425010602
42 		 0.567473627781
43 		 0.556581554828
44 		 0.546545829848
45 		 0.537192306111
46 		 0.52837026668
47 		 0.51994778702


In [38]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[  9.99999889   9.99999926   9.99999949  10.00000145]
 [  9.99998842   3.99999282   8.99999495  10.00001465]
 [  7.99999505   8.99999708   9.99999797   5.00000622]
 [  8.99998792   7.99999301   9.99999517  10.00001509]
 [ 10.00000607   5.00000415   9.0000031    8.99999249]
 [  5.9999909    3.99999466   9.99999633   6.00001151]
 [  9.00000643   8.00000442  10.00000327   8.99999197]
 [ 10.00000201   5.00000143   9.00000113   7.99999764]
 [  6.99999913   7.99999973   9.99999991   8.00000113]
 [  9.00000747   5.00000466   9.00000337   6.99999082]
 [  9.00000526   8.00000329  10.00000237   7.9999935 ]
 [  8.99999981   9.99999996   9.99999999   9.00000025]
 [  9.99999985   8.9999999    9.99999994   8.00000025]
 [  5.00000207   8.00000147   5.00000105   7.99999728]
 [ 10.00000703   8.00000422  10.00000298   9.99999132]
 [  8.99999444   8.99999646   9.99999748  10.00000702]
 [  9.00001158   8.00000693   8.00000489   7.99998561]
 [  9.99999557   7.99999718   0.99999793  10.0000054 ]
 [  5.0000

**Write your summary here.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [39]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[ nan,  10.,  10.,  10.],
        [ 10.,   4.,   9.,  10.],
        [  8.,   9.,  10.,   5.],
        [  9.,   8.,  10.,  10.],
        [ 10.,   5.,   9.,   9.],
        [  6.,   4.,  10.,   6.],
        [  9.,   8.,  10.,   9.],
        [ 10.,   5.,   9.,   8.],
        [  7.,   8.,  10.,   8.],
        [  9.,   5.,   9.,   7.],
        [  9.,   8.,  10.,   8.],
        [  9.,  10.,  10.,   9.],
        [ 10.,   9.,  10.,   8.],
        [  5.,   8.,   5.,   8.],
        [ 10.,   8.,  10.,  10.],
        [  9.,   9.,  10.,  10.],
        [  9.,   8.,   8.,   8.],
        [ 10.,   8.,   1.,  10.],
        [  5.,   6.,  10.,  10.],
        [  8.,   7.,  10.,   7.]])

In [41]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat,4,0.005,250)#use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 43.639677309
2 		 15.6067374851
3 		 4.45284055877
4 		 2.94877210209
5 		 2.75748835172
6 		 2.69524699907
7 		 2.65329431496
8 		 2.61272354841
9 		 2.56898661775
10 		 2.52062837546
11 		 2.46691842982
12 		 2.40739397503
13 		 2.34181610211
14 		 2.27020866298
15 		 2.19290322122
16 		 2.1105652677
17 		 2.02418715778
18 		 1.93503817033
19 		 1.8445696442
20 		 1.75428385848
21 		 1.66558651551
22 		 1.57965013637
23 		 1.49731566323
24 		 1.41905104039
25 		 1.34497115496
26 		 1.27490849906
27 		 1.20851349597
28 		 1.1453603635
29 		 1.08503812723
30 		 1.02721412974
31 		 0.971665837911
32 		 0.918283603279
33 		 0.867051322943
34 		 0.818013769776
35 		 0.771239276068
36 		 0.726785096255
37 		 0.684670677239
38 		 0.644861672006
39 		 0.607265231141
40 		 0.571735224444
41 		 0.53808480349
42 		 0.506103189034
43 		 0.475573674375
44 		 0.446290386498
45 		 0.418072110572
46 		 0.390772257427
47 		 0.364284703544


In [42]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 11.820448219478564:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [43]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat =FunkSVD(first_1000_users,4,0.005,20) #fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 23.2316749924
2 		 10.7475814657
3 		 7.39410259799
4 		 5.72044732782
5 		 4.66199124604
6 		 3.92010005299
7 		 3.36763109923
8 		 2.93950214069
9 		 2.59822286673
10 		 2.32026243761
11 		 2.09001737271
12 		 1.89677988481
13 		 1.73298571689
14 		 1.59311237862
15 		 1.47297779839
16 		 1.36930404059
17 		 1.27945078634
18 		 1.20125084519
19 		 1.13290438131
20 		 1.07290684181


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [53]:
# Replace each of the comments below with the correct values
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))# How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1]-num_ratings# How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [54]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
